### Convolutions (Naive, MM, im2col, Winograd)
#### Naive

In [3]:
import numpy as np

input_matrix = np.array([[3., 9., 0.], [2., 8., 1.], [1., 4., 8.]])
kernel = np.array([[8., 9.], [4., 4.]])

bias = np.array([0.06])



In [13]:
# Naive 2D conv
def conv_2d(x, kernel, bias):
    kernel_shape = kernel.shape[0]
    
    # Assuming Padding = 0, stride = 1
    output_shape = x.shape[0] - kernel_shape + 1
    result = np.zeros((output_shape, output_shape))
    
    for row in range(x.shape[0] - 1):
        for col in range(x.shape[1] - 1):
            window = x[row: row + kernel_shape, col: col + kernel_shape]
            result[row, col] = np.sum(np.multiply(kernel, window))
    return result + bias



In [14]:
print(conv_2d(input_matrix, kernel, bias))

[[145.06 108.06]
 [108.06 121.06]]


#### Conv2D with im2col 

In [19]:
# im2col: Image Block to Column

def naive_im2col(x, kernel):
    kernel_shape = kernel.shape[0]
    rows = []
    
    # Assuming Padding = 0, stride = 1
    for row in range(x.shape[0] - 1):
        for col in range(x.shape[1] - 1):
            window = x[row: row + kernel_shape, col: col + kernel_shape]
            rows.append(window.flatten())
            
    return np.transpose(np.array(rows))

In [20]:
# Conv2d with im2col

output_shape = (input_matrix.shape[0] - kernel.shape[0]) + 1

im2col_matrix = naive_im2col(input_matrix, kernel) 
print(im2col_matrix)

# np.dot exploits the vectorization of operations, still not the best case
im2col_conv = np.dot(kernel.flatten(), im2col_matrix) + bias
im2col_conv = im2col_conv.reshape(output_shape,output_shape)
print(im2col_conv)

[[3. 9. 2. 8.]
 [9. 0. 8. 1.]
 [2. 8. 1. 4.]
 [8. 1. 4. 8.]]
[[145.06 108.06]
 [108.06 121.06]]


In [38]:
# np memory stride
x = np.array([[1,2,3], [4,5,6], [7,8,9]])
print(x)

# instead of jumping 12 bytes (3 elements) to start the next row, 
# we used 'as_strided' to jump only 4 bytes (1 element)
x_newview = np.lib.stride_tricks.as_strided(x, shape = (5, 4), strides = (4, 4))

print(x_newview)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 2 3 4]
 [2 3 4 5]
 [3 4 5 6]
 [4 5 6 7]
 [5 6 7 8]]


In [39]:
x_newview[1,3] = -99
print(x_newview)
print(x)

[[  1   2   3   4]
 [  2   3   4 -99]
 [  3   4 -99   6]
 [  4 -99   6   7]
 [-99   6   7   8]]
[[  1   2   3]
 [  4 -99   6]
 [  7   8   9]]


In [8]:
from skimage.util.shape import view_as_windows

print(input_matrix)
print()
print(kernel)
print()

windows = view_as_windows(input_matrix, kernel.shape)
print(windows)
print()

# reshape
output_shape = (input_matrix.shape[0] - kernel.shape[0]) + 1
windows = windows.reshape(output_shape**2, kernel.shape[0]*2)
print(windows)

[[3. 9. 0.]
 [2. 8. 1.]
 [1. 4. 8.]]

[[8. 9.]
 [4. 4.]]

[[[[3. 9.]
   [2. 8.]]

  [[9. 0.]
   [8. 1.]]]


 [[[2. 8.]
   [1. 4.]]

  [[8. 1.]
   [4. 8.]]]]

[[3. 9. 2. 8.]
 [9. 0. 8. 1.]
 [2. 8. 1. 4.]
 [8. 1. 4. 8.]]


In [10]:
# Final function
def memory_strided_im2col(x, kernel):
    output_shape = (x.shape[0] - kernel.shape[0]) + 1
    return view_as_windows(x, kernel.shape).reshape(output_shape*output_shape, kernel.shape[0]*2)

In [11]:
output_shape = (input_matrix.shape[0] - kernel.shape[0]) + 1
mem_strided_mat = memory_strided_im2col(input_matrix, kernel) 

mem_strided_conv = np.dot(kernel.flatten(), mem_strided_mat) + bias
mem_strided_conv = mem_strided_conv.r

# ref
# https://towardsdatascience.com/how-are-convolutions-actually-performed-under-the-hood-226523ce7fbf


[[145.06 108.06]
 [108.06 121.06]]


### Winograd

In [12]:
# def winograd(im2coled_input, kernel):
# https://medium.com/@dmangla3/understanding-winograd-fast-convolution-a75458744ff
    

SyntaxError: incomplete input (3338453150.py, line 2)